In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format
import json
import csv

In [2]:
start_date = '2021-01-01'
final_date = '2022-02-28'

### PARAMETRIA COMISIONES

In [3]:
parametria_comisiones = pd.read_csv(r'G:\My Drive\RESPALDO-PC\TOPUPS\PARAMETRIA TOPUPS.csv',dtype={'monto_recarga':float})
parametria_comisiones['inicio'] = pd.to_datetime(parametria_comisiones['inicio'])
parametria_comisiones['porcentaje_comision_final'] = parametria_comisiones['porcentaje_comision']/100
parametria_comisiones['index_comisiones'] = (parametria_comisiones['carrier'].astype(str).replace(' ', '',regex=True)+'_'
                                             +parametria_comisiones['monto_recarga'].astype(str))
parametria_comisiones.head()

,motivo_del_cambio,naturaleza,estatus,inicio,fin,carrier,monto_recarga,porcentaje_comision,categoria,porcentaje_comision_final,index_comisiones
0,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,30.00,7.00,MOVISTAR,0.07,MOVISTAR_30.0
1,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,50.00,7.00,MOVISTAR,0.07,MOVISTAR_50.0
2,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,100.00,7.00,MOVISTAR,0.07,MOVISTAR_100.0
3,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,200.00,7.00,MOVISTAR,0.07,MOVISTAR_200.0
4,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,500.00,7.00,MOVISTAR,0.07,MOVISTAR_500.0


In [4]:
parametria_comisiones.shape

(26, 11)

In [5]:
parametria_comisiones.dtypes

motivo_del_cambio                    object
naturaleza                           object
estatus                              object
inicio                       datetime64[ns]
fin                                  object
carrier                              object
monto_recarga                       float64
porcentaje_comision                 float64
categoria                            object
porcentaje_comision_final           float64
index_comisiones                     object
dtype: object

In [6]:
parametria_comisiones['carrier'].unique()

array(['MOVISTAR', 'UNEFON', 'TELCEL', 'ATT', 'IUSACELL'], dtype=object)

In [7]:
parametria_comisiones['index_comisiones'].unique()

array(['MOVISTAR_30.0', 'MOVISTAR_50.0', 'MOVISTAR_100.0',
       'MOVISTAR_200.0', 'MOVISTAR_500.0', 'UNEFON_30.0', 'UNEFON_50.0',
       'UNEFON_100.0', 'UNEFON_200.0', 'UNEFON_500.0', 'TELCEL_30.0',
       'TELCEL_50.0', 'TELCEL_80.0', 'TELCEL_100.0', 'TELCEL_200.0',
       'TELCEL_500.0', 'ATT_30.0', 'ATT_50.0', 'ATT_100.0', 'ATT_200.0',
       'ATT_500.0', 'IUSACELL_30.0', 'IUSACELL_50.0', 'IUSACELL_100.0',
       'IUSACELL_200.0', 'IUSACELL_500.0'], dtype=object)

### SERTI (EXTRAIDO EN EXCEL)

In [8]:
serti_files = glob.glob(r'G:\My Drive\RESPALDO-PC\TOPUPS\202*\**\SERTI\*.csv',recursive=False)
serti_files


['G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (1).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (2).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (3).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (4).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (5).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (6).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (7).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (8).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (9).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (10).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (11).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (12).csv',


In [9]:
serti = pd.concat([pd.read_csv(f,dtype={'bin':object,'panTerminacion':object,'claveRechazo':object,'phone':object
                                        ,'transactionId':object,'importe':float})
                   .assign(file_name=os.path.basename(f)) for f in serti_files])
# serti.loc[serti['carrier'].str.contains('AT&T'), 'carrier'] = 'ATT'
serti['carrier'] = serti['carrier'].str.strip()
serti['carrier'] = serti['carrier'].str.upper()
serti['last4_phone_serti'] = serti['phone'].str[-4:]
serti['serti_date'] = serti['date'].str[:10]
serti['serti_date'] = pd.to_datetime(serti['serti_date'])
serti.sort_values(by=['date'],inplace=True)
serti['INDEX_TOTAL'] = (serti['transactionId'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['last4_phone_serti'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['panTerminacion'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['importe'].astype(str).replace(' ', '',regex=True))
# serti['index_comisiones'] = (serti['carrier'].astype(str).replace(' ', '',regex=True) + '_'
#                        + serti['importe'].astype(str).replace(' ', '',regex=True))
serti.reset_index(inplace=True)
serti.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,comision,ivaComision,carrier,phone,folio,file_name,claveVenta,last4_phone_serti,serti_date,INDEX_TOTAL
0,7954,402766,7364,50.00,2021-05-01T00:00:21.918,34205977,Chip,"482,732.00",NaN,0,...,0,0,UNEFON,*******4162,"502,474.00",result.csv,NaN,4162,2021-05-01,34205977_4162_7364_50.0
1,7953,415231,5366,100.00,2021-05-01T00:01:24.31,34206011,Chip,"318,315.00",NaN,0,...,0,0,TELCEL,*******0854,"458,139.00",result.csv,NaN,0854,2021-05-01,34206011_0854_5366_100.0
2,7952,415231,7946,100.00,2021-05-01T00:02:20.669,34206035,Chip,"321,173.00",NaN,0,...,0,0,TELCEL,*******8827,"143,713.00",result.csv,NaN,8827,2021-05-01,34206035_8827_7946_100.0
3,7951,412408,2697,50.00,2021-05-01T00:03:08.02,34206062,Chip,"649,428.00",NaN,0,...,0,0,TELCEL,*******7918,"19,708.00",result.csv,NaN,7918,2021-05-01,34206062_7918_2697_50.0
4,7950,404313,3500,50.00,2021-05-01T00:04:12.187,34206091,Chip,"863,609.00",NaN,0,...,0,0,IUSACELL,*******1258,"503,435.00",result.csv,NaN,1258,2021-05-01,34206091_1258_3500_50.0


In [10]:
serti.shape

(2276199, 25)

In [11]:
serti.columns

Index(['index', 'bin', 'panTerminacion', 'importe', 'date', 'transactionId',
       'entryMode', 'numAuth', 'terminal', 'caja', 'aprobada', 'claveRechazo',
       'merchant', 'producto', 'tarjetahabiente', 'comision', 'ivaComision',
       'carrier', 'phone', 'folio', 'file_name', 'claveVenta',
       'last4_phone_serti', 'serti_date', 'INDEX_TOTAL'],
      dtype='object')

In [12]:
serti['index_comisiones'].unique()

KeyError: 'index_comisiones'

In [13]:
duplicates_serti = serti[serti.duplicated(['transactionId'] , keep=False)]
duplicates_serti.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,comision,ivaComision,carrier,phone,folio,file_name,claveVenta,last4_phone_serti,serti_date,INDEX_TOTAL
353733,6405,421316,1149,100.00,2021-06-20T00:00:13.774,37812948,Chip,"57,708.00",NaN,0,...,0,0,UNEFON,9842888311,804866,result (19).csv,NaN,8311,2021-06-20,37812948_8311_1149_100.0
353734,6405,421316,1149,100.00,2021-06-20T00:00:13.774,37812948,Chip,"57,708.00",NaN,0,...,0,0,UNEFON,9842888311,804866,result (18).csv,NaN,8311,2021-06-20,37812948_8311_1149_100.0
353735,6408,415231,7199,50.00,2021-06-20T00:00:27.099,37812912,Chip,"220,282.00",NaN,0,...,0,0,UNEFON,8993335289,804835,result (19).csv,NaN,5289,2021-06-20,37812912_5289_7199_50.0
353736,6409,415231,7199,50.00,2021-06-20T00:00:27.099,37812912,Chip,"220,282.00",NaN,0,...,0,0,UNEFON,8993335289,804835,result (18).csv,NaN,5289,2021-06-20,37812912_5289_7199_50.0
353737,6404,520416,0590,30.00,2021-06-20T00:00:31.474,37812952,Chip,"19,801.00",NaN,0,...,0,0,TELCEL,8445332124,731780,result (18).csv,NaN,2124,2021-06-20,37812952_2124_0590_30.0


In [14]:
duplicates_serti.shape

(12818, 25)

In [15]:
serti.drop_duplicates(subset='INDEX_TOTAL', keep='first', inplace=True)

In [16]:
serti.shape

(2269790, 25)

In [17]:
serti.dtypes

index                         int64
bin                          object
panTerminacion               object
importe                     float64
date                         object
transactionId                object
entryMode                    object
numAuth                      object
terminal                    float64
caja                          int64
aprobada                       bool
claveRechazo                 object
merchant                      int64
producto                     object
tarjetahabiente              object
comision                      int64
ivaComision                   int64
carrier                      object
phone                        object
folio                        object
file_name                    object
claveVenta                  float64
last4_phone_serti            object
serti_date           datetime64[ns]
INDEX_TOTAL                  object
dtype: object

In [18]:
serti_period =  serti[(serti['serti_date'] >= start_date) & (serti['serti_date'] <= final_date)]
serti_period['serti_date'].unique()

array(['2021-05-01T00:00:00.000000000', '2021-05-02T00:00:00.000000000',
       '2021-05-03T00:00:00.000000000', '2021-05-04T00:00:00.000000000',
       '2021-05-05T00:00:00.000000000', '2021-05-06T00:00:00.000000000',
       '2021-05-07T00:00:00.000000000', '2021-05-08T00:00:00.000000000',
       '2021-05-09T00:00:00.000000000', '2021-05-10T00:00:00.000000000',
       '2021-05-11T00:00:00.000000000', '2021-05-12T00:00:00.000000000',
       '2021-05-13T00:00:00.000000000', '2021-05-14T00:00:00.000000000',
       '2021-05-15T00:00:00.000000000', '2021-05-16T00:00:00.000000000',
       '2021-05-17T00:00:00.000000000', '2021-05-18T00:00:00.000000000',
       '2021-05-19T00:00:00.000000000', '2021-05-20T00:00:00.000000000',
       '2021-05-21T00:00:00.000000000', '2021-05-22T00:00:00.000000000',
       '2021-05-23T00:00:00.000000000', '2021-05-24T00:00:00.000000000',
       '2021-05-25T00:00:00.000000000', '2021-05-26T00:00:00.000000000',
       '2021-05-27T00:00:00.000000000', '2021-05-28

### SERTI CON PARAMETRÍA DE COMISIONES

In [21]:
serti_final = serti_period.merge(parametria_comisiones[['index_comisiones','estatus','porcentaje_comision_final']]
                      ,on='index_comisiones',how='left')
serti_final['estatus'].fillna('Not_found',inplace=True)
serti_final['porcentaje_comision_final'].fillna(0,inplace=True)
serti_final['comision_db'] = serti_final['importe'] * serti_final['porcentaje_comision_final']
serti_final['iva_comision_db'] = serti_final['comision_db'] * 0.1600
serti_final.head()

KeyError: 'index_comisiones'

In [22]:
serti_final.shape

NameError: name 'serti_final' is not defined

### PIVOT SERTI

In [19]:
pivot_serti = pd.pivot_table(serti_final,index=['serti_date']
                            ,columns=['aprobada']
                            ,values=['importe','transactionId','comision','ivaComision','comision_db','iva_comision_db']
                            ,aggfunc={'importe':np.sum,
                                     'transactionId':len,
                                      'comision':np.sum,
                                      'ivaComision':np.sum,
                                     'comision_db':np.sum,
                                     'iva_comision_db':np.sum,},margins=True).reset_index()
pivot_serti.fillna(0,inplace=True)
pivot_serti.head()

NameError: name 'serti_final' is not defined

In [20]:
pivot_serti.shape

NameError: name 'pivot_serti' is not defined

### DB PRODUCTO

In [23]:
producto_files = glob.glob(r'G:\My Drive\RESPALDO-PC\TOPUPS\202*\**\DB\TOPUPS*.csv',recursive=False)
producto_files


['G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\DB\\TOPUPS MAYO 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\06. JUNIO\\DB\\TOPUPS JUNIO 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\07. JULIO\\DB\\TOPUPS JULIO 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\09. SEPTIEMBRE\\DB\\TOPUPS SEPTIEMBRE 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\08. AGOSTO\\DB\\TOPUPS AGOSTO 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\11. NOVIEMBRE\\DB\\TOPUPS OCTUBRE - NOVIEMBRE 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\12. DICIEMBRE\\DB\\TOPUPS DICIEMBRE 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\DB\\TOPUPS ENERO 2022.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\02. FEBRERO\\DB\\TOPUPS FEBRERO 2022.csv']

In [24]:
topups_db = pd.concat([pd.read_csv(f,dtype={'clip_external_tracking_id':object,'serti_transaction_id':object,
                                            'phone_number':object,'last4':object,'amount':float,'status_code':object})
                   .assign(file_name=os.path.basename(f)) for f in producto_files])
topups_db['clip_external_tracking_id'].fillna('sin_order_id_db',inplace=True)
topups_db['serti_transaction_id'].fillna('sin_order_id_db',inplace=True)
topups_db['created_at'] = pd.to_datetime(topups_db['created_at'])
topups_db['updated_at'] = pd.to_datetime(topups_db['updated_at'])
topups_db['last4_phone'] = topups_db['phone_number'].str[-4:]
topups_db['INDEX_TOTAL'] = (topups_db['serti_transaction_id'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4_phone'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['amount'].astype(str).replace(' ', '',regex=True))
topups_db['index_comisiones'] = (topups_db['carrier'].astype(str).replace(' ', '',regex=True)+'-'
                                             +topups_db['amount'].astype(str))
topups_db.head()

C:\Users\CRISTI~1.APA\AppData\Local\Temp/ipykernel_7268/1884929116.py:1: DtypeWarning: Columns (15,19) have mixed types.Specify dtype option on import or set low_memory=False.
  topups_db = pd.concat([pd.read_csv(f,dtype={'clip_external_tracking_id':object,'serti_transaction_id':object,


,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier,phone_number,date_created,created_at,payment_type,status_code,...,file_name,date_created_mexico_city,created_at_mexico_city,updated_at,created_at_datetime,updated_at_datetime,serti_transaction_id,last4_phone,INDEX_TOTAL,index_comisiones
0,588f8153-01f9-4ccd-ae99-34be495f3caa,e0d19424-484f-4a22-97ff-bce7d79f0c63,ff80481a-afb6-452c-9821-451268396ced,34205977,UNEFON,5621454162,2021-05-01,2021-05-01 05:00:20,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,4162,sin_order_id_db_4162_7364_50.0,UNEFON-50.0
1,e8b3d057-dc8d-4f0c-8687-754a1c7b84a5,422b5ffc-82ed-4b33-8374-477915a34574,54c6e184-d9a6-43e3-820b-be8c983d1aac,34206011,TELCEL,5510080854,2021-05-01,2021-05-01 05:01:22,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,0854,sin_order_id_db_0854_5366_100.0,TELCEL-100.0
2,33e06d50-9718-42dd-b213-96a923414b1f,3164f6fc-dbf2-49b6-b7d3-3c20576c0d10,672af042-e82d-42d9-b566-ad6660633592,34206035,TELCEL,6648088827,2021-05-01,2021-05-01 05:02:19,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,8827,sin_order_id_db_8827_7946_100.0,TELCEL-100.0
3,12c27a23-a6ad-4a59-94ba-2611fd04aa8c,c3e69929-3222-4875-8f7f-8f9204546f49,502276d4-964d-4dd0-8a9b-6677c0298eef,34206062,TELCEL,4494897918,2021-05-01,2021-05-01 05:03:06,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,7918,sin_order_id_db_7918_2697_50.0,TELCEL-50.0
4,54d9a6e8-9b43-4084-a141-3706985723b7,4581385b-e1e3-4642-8355-8cb2fc25db35,1875f768-b7fd-4623-bd48-401157b2c70e,34206091,ATT,3320491258,2021-05-01,2021-05-01 05:04:10,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,1258,sin_order_id_db_1258_3500_50.0,ATT-50.0


In [25]:
topups_db.shape

(2167682, 31)

In [26]:
topups_db['carrier'].unique()

array(['UNEFON', 'TELCEL', 'ATT', 'MOVISTAR'], dtype=object)

In [27]:
duplicates = topups_db[topups_db.duplicated(['serti_transaction_id'], keep=False)]
duplicates.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier,phone_number,date_created,created_at,payment_type,status_code,...,file_name,date_created_mexico_city,created_at_mexico_city,updated_at,created_at_datetime,updated_at_datetime,serti_transaction_id,last4_phone,INDEX_TOTAL,index_comisiones
0,588f8153-01f9-4ccd-ae99-34be495f3caa,e0d19424-484f-4a22-97ff-bce7d79f0c63,ff80481a-afb6-452c-9821-451268396ced,34205977,UNEFON,5621454162,2021-05-01,2021-05-01 05:00:20,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,4162,sin_order_id_db_4162_7364_50.0,UNEFON-50.0
1,e8b3d057-dc8d-4f0c-8687-754a1c7b84a5,422b5ffc-82ed-4b33-8374-477915a34574,54c6e184-d9a6-43e3-820b-be8c983d1aac,34206011,TELCEL,5510080854,2021-05-01,2021-05-01 05:01:22,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,0854,sin_order_id_db_0854_5366_100.0,TELCEL-100.0
2,33e06d50-9718-42dd-b213-96a923414b1f,3164f6fc-dbf2-49b6-b7d3-3c20576c0d10,672af042-e82d-42d9-b566-ad6660633592,34206035,TELCEL,6648088827,2021-05-01,2021-05-01 05:02:19,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,8827,sin_order_id_db_8827_7946_100.0,TELCEL-100.0
3,12c27a23-a6ad-4a59-94ba-2611fd04aa8c,c3e69929-3222-4875-8f7f-8f9204546f49,502276d4-964d-4dd0-8a9b-6677c0298eef,34206062,TELCEL,4494897918,2021-05-01,2021-05-01 05:03:06,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,7918,sin_order_id_db_7918_2697_50.0,TELCEL-50.0
4,54d9a6e8-9b43-4084-a141-3706985723b7,4581385b-e1e3-4642-8355-8cb2fc25db35,1875f768-b7fd-4623-bd48-401157b2c70e,34206091,ATT,3320491258,2021-05-01,2021-05-01 05:04:10,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,1258,sin_order_id_db_1258_3500_50.0,ATT-50.0


In [28]:
duplicates.shape

(1194621, 31)

In [29]:
duplicates['serti_transaction_id'].unique()

array(['sin_order_id_db', '0'], dtype=object)

In [30]:
topups_db.shape

(2167682, 31)

In [31]:
topups_db.dtypes

merchant_id                          object
topup_id                             object
transaction_id                       object
clip_external_tracking_id            object
carrier                              object
phone_number                         object
date_created                         object
created_at                   datetime64[ns]
payment_type                         object
status_code                          object
status_msg                           object
amount                              float64
processor                            object
issuer_auth_code                     object
folio_id                            float64
receipt_no                           object
last4                                object
authNum                              object
orderId                               int64
type                                 object
response_code                        object
file_name                            object
date_created_mexico_city        

### DB CON PARAMETRÍA DE COMISIONES

In [32]:
topups_db_final = topups_db

In [33]:
# topups_db_final = topups_db.merge(parametria_comisiones[['index_comisiones','estatus','porcentaje_comision_final']]
#                       ,on='index_comisiones',how='left')
# topups_db_final['estatus'].fillna('Not_found',inplace=True)
# topups_db_final['porcentaje_comision_final'].fillna(0,inplace=True)
# topups_db_final['comision_db'] = topups_db_final['amount'] * topups_db_final['porcentaje_comision_final']
# topups_db_final['iva_comision_db'] = topups_db_final['comision_db'] * 0.1600
# topups_db_final.head()

In [34]:
topups_db_final.shape

(2167682, 31)

### SERTI VS DB

In [36]:
cruce_1 = serti.merge(topups_db_final[['INDEX_TOTAL','topup_id','transaction_id','clip_external_tracking_id',
                                             'serti_transaction_id','created_at','updated_at','status_code',
                                             'status_msg','amount','last4_phone','carrier','type',
                                             'response_code']]
                      ,on='INDEX_TOTAL',how='left')
cruce_1['status_code'].fillna('Not_found',inplace=True)
cruce_1['topup_id'].fillna('Not_found',inplace=True)
cruce_1.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,serti_transaction_id,created_at,updated_at,status_code,status_msg,amount,last4_phone,carrier_y,type,response_code
0,7954,402766,7364,50.00,2021-05-01T00:00:21.918,34205977,Chip,"482,732.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,NaN,NaN,NaN,NaN,NaN
1,7953,415231,5366,100.00,2021-05-01T00:01:24.31,34206011,Chip,"318,315.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,NaN,NaN,NaN,NaN,NaN
2,7952,415231,7946,100.00,2021-05-01T00:02:20.669,34206035,Chip,"321,173.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,NaN,NaN,NaN,NaN,NaN
3,7951,412408,2697,50.00,2021-05-01T00:03:08.02,34206062,Chip,"649,428.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,NaN,NaN,NaN,NaN,NaN
4,7950,404313,3500,50.00,2021-05-01T00:04:12.187,34206091,Chip,"863,609.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
cruce_1.shape

(2269790, 38)

In [38]:
cruce_1['aprobada'].unique()

array([ True, False])

### TRUE ONLY

In [39]:
cruce_1_true = cruce_1[(cruce_1['aprobada'] == True)]
cruce_1_true['amount'].fillna(0,inplace=True)
cruce_1_true.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,serti_transaction_id,created_at,updated_at,status_code,status_msg,amount,last4_phone,carrier_y,type,response_code
0,7954,402766,7364,50.00,2021-05-01T00:00:21.918,34205977,Chip,"482,732.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,0.00,NaN,NaN,NaN,NaN
1,7953,415231,5366,100.00,2021-05-01T00:01:24.31,34206011,Chip,"318,315.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,0.00,NaN,NaN,NaN,NaN
2,7952,415231,7946,100.00,2021-05-01T00:02:20.669,34206035,Chip,"321,173.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,0.00,NaN,NaN,NaN,NaN
3,7951,412408,2697,50.00,2021-05-01T00:03:08.02,34206062,Chip,"649,428.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,0.00,NaN,NaN,NaN,NaN
4,7950,404313,3500,50.00,2021-05-01T00:04:12.187,34206091,Chip,"863,609.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,0.00,NaN,NaN,NaN,NaN


In [40]:
cruce_1_true.shape

(2093220, 38)

In [41]:
duplicates_cruce_1_true = cruce_1_true[cruce_1_true.duplicated(['INDEX_TOTAL'] , keep=False)]
duplicates_cruce_1_true.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,serti_transaction_id,created_at,updated_at,status_code,status_msg,amount,last4_phone,carrier_y,type,response_code


In [42]:
duplicates_cruce_1_true.shape

(0, 38)

In [43]:
cruce_1_final = cruce_1_true[(cruce_1_true['topup_id'] != 'Not_found')]
cruce_1_final.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,serti_transaction_id,created_at,updated_at,status_code,status_msg,amount,last4_phone,carrier_y,type,response_code
1055658,8958,455511,1200,200.00,2021-09-27T04:08:51.295,44494195,Chip,628308,NaN,0,...,44494195,2021-09-27,2021-10-04,1,APPROVED,200.00,8819,TELCEL,emv_sale,00
1055660,8956,416916,0940,30.00,2021-09-27T04:11:14.972,44494203,Chip,384843,NaN,0,...,44494203,2021-09-27,2021-10-04,1,APPROVED,30.00,4878,ATT,emv_sale,00
1055826,8790,416916,0549,100.00,2021-09-27T07:29:43.919,44495958,Chip,415642,NaN,0,...,44495958,2021-09-27,2021-10-04,1,APPROVED,100.00,6162,TELCEL,emv_sale,00
1055863,8753,402766,7051,200.00,2021-09-27T07:40:41.156,44496181,Chip,063965,NaN,0,...,44496181,2021-09-27,2021-10-04,1,APPROVED,200.00,1106,TELCEL,emv_sale,00
1055894,8722,420199,9562,50.00,2021-09-27T07:48:27.56,44496363,Chip,057082,NaN,0,...,44496363,2021-09-27,2021-10-04,1,APPROVED,50.00,8157,UNEFON,emv_sale,00


In [44]:
cruce_1_final.shape

(886224, 38)

In [45]:
na_cruce_1_final = cruce_1_true[(cruce_1_true['topup_id'] == 'Not_found')]
na_cruce_1_final.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,serti_transaction_id,created_at,updated_at,status_code,status_msg,amount,last4_phone,carrier_y,type,response_code
0,7954,402766,7364,50.00,2021-05-01T00:00:21.918,34205977,Chip,"482,732.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,0.00,NaN,NaN,NaN,NaN
1,7953,415231,5366,100.00,2021-05-01T00:01:24.31,34206011,Chip,"318,315.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,0.00,NaN,NaN,NaN,NaN
2,7952,415231,7946,100.00,2021-05-01T00:02:20.669,34206035,Chip,"321,173.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,0.00,NaN,NaN,NaN,NaN
3,7951,412408,2697,50.00,2021-05-01T00:03:08.02,34206062,Chip,"649,428.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,0.00,NaN,NaN,NaN,NaN
4,7950,404313,3500,50.00,2021-05-01T00:04:12.187,34206091,Chip,"863,609.00",NaN,0,...,NaN,NaT,NaT,Not_found,NaN,0.00,NaN,NaN,NaN,NaN


In [46]:
na_cruce_1_final.shape

(1206996, 38)

In [47]:
cruce_1_true.columns

Index(['index', 'bin', 'panTerminacion', 'importe', 'date', 'transactionId',
       'entryMode', 'numAuth', 'terminal', 'caja', 'aprobada', 'claveRechazo',
       'merchant', 'producto', 'tarjetahabiente', 'comision', 'ivaComision',
       'carrier_x', 'phone', 'folio', 'file_name', 'claveVenta',
       'last4_phone_serti', 'serti_date', 'INDEX_TOTAL', 'topup_id',
       'transaction_id', 'clip_external_tracking_id', 'serti_transaction_id',
       'created_at', 'updated_at', 'status_code', 'status_msg', 'amount',
       'last4_phone', 'carrier_y', 'type', 'response_code'],
      dtype='object')

In [48]:
cruce_1.dtypes

index                                 int64
bin                                  object
panTerminacion                       object
importe                             float64
date                                 object
transactionId                        object
entryMode                            object
numAuth                              object
terminal                            float64
caja                                  int64
aprobada                               bool
claveRechazo                         object
merchant                              int64
producto                             object
tarjetahabiente                      object
comision                              int64
ivaComision                           int64
carrier_x                            object
phone                                object
folio                                object
file_name                            object
claveVenta                          float64
last4_phone_serti               

### POSIBLES RECLAMOS

In [56]:
cruce_1_posibles_reclamos = cruce_1_final[(cruce_1_final['status_code'] != '1')]
cruce_1_posibles_reclamos.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,serti_transaction_id,created_at,updated_at,status_code,status_msg,amount,last4_phone,carrier_y,type,response_code
1971113,2440,557920,1319,100.00,2022-01-11T17:22:12.091,50419584,Chip,"550,830.00",NaN,0,...,50419584,2022-01-11,2022-01-11,0,Pending,100.00,3140,TELCEL,emv_sale,00
1971114,2438,526354,7150,100.00,2022-01-11T17:22:13.832,50419588,Chip,"115,130.00",NaN,0,...,50419588,2022-01-11,2022-01-11,0,Pending,100.00,0392,ATT,emv_sale,00


In [57]:
cruce_1_posibles_reclamos.shape

(2, 38)

### PIVOT TRUE ONLY

In [ ]:
pivot_true = pd.pivot_table(cruce_1_true,index=['serti_date']
                            ,columns=['status_code']
                            ,values=['importe','amount','transactionId','serti_transaction_id','comision','ivaComision',
                                     'comision_db','iva_comision_db']
                            ,aggfunc={'importe':np.sum,
                                      'amount':np.sum,
                                      'comision':np.sum,
                                      'ivaComision':np.sum,
                                      'comision_db':np.sum,
                                      'iva_comision_db':np.sum,
                                      'transactionId':len,
                                      'serti_transaction_id':len},margins=True).reset_index(0)
pivot_true.fillna(0,inplace=True)
pivot_true.head()

In [ ]:
pivot_true.shape

In [ ]:
stacked = pivot_true.stack(level=[1]).reset_index()
stacked.head()

In [ ]:
stacked.shape

### PIVOT TRUE Y CARRIER 

In [ ]:
pivot_carrier = pd.pivot_table(cruce_1_true,index=['serti_date','carrier_x']
                            ,columns=['status_code']
                            ,values=['importe','amount','transactionId','serti_transaction_id','comision','ivaComision',
                                     'comision_db','iva_comision_db']
                            ,aggfunc={'importe':np.sum,
                                      'amount':np.sum,
                                      'comision':np.sum,
                                      'ivaComision':np.sum,
                                      'comision_db':np.sum,
                                      'iva_comision_db':np.sum,
                                      'transactionId':len,
                                      'serti_transaction_id':len},margins=True).reset_index()
pivot_carrier.fillna(0,inplace=True)
pivot_carrier.head()

In [ ]:
pivot_carrier.shape

### PIVOT RESUMEN DE COMISIONES

In [ ]:
pivot_carrier_resumen = pd.pivot_table(cruce_1_true,index=['carrier_x','serti_date']
                            ,columns=['status_code']
                            ,values=['importe','amount','transactionId','serti_transaction_id','comision','ivaComision',
                                     'comision_db','iva_comision_db']
                            ,aggfunc={'importe':np.sum,
                                      'amount':np.sum,
                                      'comision':np.sum,
                                      'ivaComision':np.sum,
                                      'comision_db':np.sum,
                                      'iva_comision_db':np.sum,
                                      'transactionId':len,
                                      'serti_transaction_id':len})
pivot_carrier_resumen.head()

In [ ]:
pivot_carrier_resumen.shape

# EXCEL FINAL

In [ ]:
writer = pd.ExcelWriter('CONCILIACION TOPUPS HISTORICO COMISIONES.xlsx', engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
pivot_serti.to_excel(writer, sheet_name='RESUMEN SERTI')
pivot_true.to_excel(writer, sheet_name='RESUMEN')
pivot_carrier.to_excel(writer, sheet_name='RESUMEN CARRIER')
pivot_carrier_resumen.to_excel(writer, sheet_name='RESUMEN COMISIONES')
cruce_1_true.to_excel(writer, sheet_name='DETALLE COMPLETO',index=False)
cruce_1_final.to_excel(writer, sheet_name='FOUND',index=False)
na_cruce_1_final.to_excel(writer, sheet_name='NOT FOUND',index=False)
duplicates_cruce_1_true.to_excel(writer, sheet_name='DUPLICADOS_INDEX_TOTAL',index=False)
writer.close()